
# Building and Optimizing a CNN

In this homework, you will design, implement, and optimize a **Convolutional Neural Network (CNN)** using PyTorch to classify images from the CIFAR-10 dataset. This will involve advanced preprocessing techniques, sophisticated model architectures, hyperparameter tuning, and comprehensive evaluation.

## Project Overview
- **Task**: Image classification on CIFAR-10 dataset
- **Architecture**: CNN with Inception bottlenecks and advanced optimizations
- **Dataset**: CIFAR-10 (60,000 32x32 color images in 10 classes)
- **Goal**: Achieve high classification accuracy with optimized training


## Import Libraries and Configuration


**Requirements**:
- Import PyTorch, torchvision, and related libraries
- Import matplotlib, numpy, and other utilities
- Set random seeds for reproducibility
- Configure hyperparameters with reasonable values

In [ ]:
!pip install torch torchvision torchmetrics matplotlib numpy scikit-learn roboflow torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
#  Import all necessary libraries:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
import copy
import time
import os

# Torch sets for reproducibility:
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Check device availability and print
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define configuration parameters:
BATCH_SIZE = 128  # Batch size for training
LEARNING_RATE = 0.001  # Initial learning rate
NUM_EPOCHS = 50  # Number of training epochs
NUM_CLASSES = 10  # CIFAR-10 has 10 classes
INPUT_SIZE = 32  # CIFAR-10 image size is 32x32

## Load and Preprocess the Data

**Task**: Load CIFAR-10 dataset and implement advanced preprocessing techniques.

**Requirements**:
- Load CIFAR-10 training and test sets
- Apply data normalization using dataset statistics
- Implement comprehensive data augmentation for training
- Create data loaders with appropriate settings

In [ ]:
# Define data transforms for training (with augmentation):
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Horizontal flip with 50% probability -> model learn orientation invariance
    transforms.RandomRotation(degrees=10),  # Random rotation within 10 degrees -> model learn rotation invariance
    transforms.RandomCrop(32, padding=4),  # Random crop with padding -> model learn translation invariance
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change brightness, contrast, saturation, and hue -> model learn color invariance
    transforms.ToTensor(),  # Convert PIL images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images with CIFAR-10 mean and std -> model learn normalized data
])

# Define data transforms for testing (no augmentation):
test_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 datasets:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Create data loaders:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# Define CIFAR-10 class names
CIFAR10_CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# Print dataset information (sizes, classes, etc.)
print(f"Number of training samples: {len(trainset)} , Number of test samples: {len(testset)}, Size of each image: {trainset[0][0].shape}, Number of classes: {len(CIFAR10_CLASSES)}")

# Visualize some sample images with their labels
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
axes = axes.flatten()

examples = iter(trainloader)
images, labels = next(examples)

# Plot images with their labels
for i in range(15):
    # Unnormalize the images for visualization
    img = images[i].permute(1, 2, 0).cpu().numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)

    axes[i].imshow(img)
    axes[i].set_title(CIFAR10_CLASSES[labels[i]])
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 3️⃣ Design Complex CNN Architecture with Inception Bottlenecks

**Task**: Implement a sophisticated CNN architecture incorporating Inception-style bottleneck blocks.

**Requirements**:
- Create an Inception bottleneck module with multiple parallel paths
- Design the main CNN with multiple Inception blocks
- Use appropriate pooling, batch normalization, and dropout
- Implement skip connections where beneficial

In [ ]:
# Create InceptionBottleneck class inheriting from nn.Module
class InceptionBottleneck(nn.Module):
    """
    Inception-style bottleneck module with four parallel paths:
    1. 1x1 convolution
    2. 1x1 reduction -> 3x3 convolution
    3. 1x1 reduction -> 5x5 convolution
    4. 3x3 max pooling -> 1x1 projection
    """

    def __init__(self, in_channels, out_1x1, reduce_3x3, out_3x3, reduce_5x5, out_5x5, out_pool):
        super(InceptionBottleneck, self).__init__()

        # Path 1: 1x1 convolution
        self.branch1x1 = nn.Sequential(
            nn.Conv2d(in_channels, out_1x1, kernel_size=1),
            nn.BatchNorm2d(out_1x1),
            nn.ReLU(inplace=True)
        )

        # Path 2: 1x1 reduction + 3x3 convolution
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, reduce_3x3, kernel_size=1),
            nn.BatchNorm2d(reduce_3x3),
            nn.ReLU(inplace=True),
            nn.Conv2d(reduce_3x3, out_3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_3x3),
            nn.ReLU(inplace=True)
        )

        # Path 3: 1x1 reduction + 5x5 convolution
        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, reduce_5x5, kernel_size=1),
            nn.BatchNorm2d(reduce_5x5),
            nn.ReLU(inplace=True),
            nn.Conv2d(reduce_5x5, out_5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(out_5x5),
            nn.ReLU(inplace=True)
        )

        # Path 4: 3x3 max pooling + 1x1 projection
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool, kernel_size=1),
            nn.BatchNorm2d(out_pool),
            nn.ReLU(inplace=True)
        )

    # Forward method for InceptionBottleneck:
    def forward(self, x):
        # Pass input through all four paths
        branch1x1_out = self.branch1x1(x)
        branch3x3_out = self.branch3x3(x)
        branch5x5_out = self.branch5x5(x)
        branch_pool_out = self.branch_pool(x)

        # Concatenate outputs along channel dimension
        outputs = torch.cat([branch1x1_out, branch3x3_out, branch5x5_out, branch_pool_out], dim=1)
        return outputs

class InceptionCNN(nn.Module):
    """
    CNN architecture with Inception bottleneck blocks designed for CIFAR-10 classification (32x32 RGB images)
    """
    def __init__(self, num_classes=10):
        super(InceptionCNN, self).__init__()

        # Initial convolutional layers
        self.initial_conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 32x32 -> 16x16
            nn.Dropout2d(0.1)
        )

        # First set of Inception blocks
        self.inception1 = InceptionBottleneck(
            in_channels=64,
            out_1x1=16,
            reduce_3x3=32, out_3x3=64,
            reduce_5x5=16, out_5x5=32,
            out_pool=32
        )  # Output channels: 16 + 64 + 32 + 32 = 144

        self.inception2 = InceptionBottleneck(
            in_channels=144,
            out_1x1=32,
            reduce_3x3=64, out_3x3=128,
            reduce_5x5=32, out_5x5=64,
            out_pool=64
        )  # Output channels: 32 + 128 + 64 + 64 = 288

        self.pool1 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),  # 16x16 -> 8x8
            nn.Dropout2d(0.2)
        )

        # Second set of Inception blocks
        self.inception3 = InceptionBottleneck(
            in_channels=288,
            out_1x1=64,
            reduce_3x3=128, out_3x3=256,
            reduce_5x5=64, out_5x5=128,
            out_pool=128
        )  # Output channels: 64 + 256 + 128 + 128 = 576

        self.inception4 = InceptionBottleneck(
            in_channels=576,
            out_1x1=128,
            reduce_3x3=256, out_3x3=512,
            reduce_5x5=128, out_5x5=256,
            out_pool=256
        )  # Output channels: 128 + 512 + 256 + 256 = 1152

        # Global average pooling instead of large fully connected layers
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1152, num_classes)
        )

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize network weights using Xavier/He initialization"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Initial convolution layers
        x = self.initial_conv(x)  # (batch, 64, 16, 16)

        # First set of Inception blocks
        x = self.inception1(x)    # (batch, 144, 16, 16)
        x = self.inception2(x)    # (batch, 288, 16, 16)
        x = self.pool1(x)         # (batch, 288, 8, 8)

        # Second set of Inception blocks
        x = self.inception3(x)    # (batch, 576, 8, 8)
        x = self.inception4(x)    # (batch, 1152, 8, 8)

        # Global average pooling
        x = self.global_avg_pool(x)  # (batch, 1152, 1, 1)

        # Flatten for classifier
        x = x.view(x.size(0), -1)   # (batch, 1152)

        # Classification
        x = self.classifier(x)      # (batch, num_classes)

        return x

    def count_parameters(self):
        """Count total number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Initialize the model
model = InceptionCNN(num_classes=10).to(device)
print(f"Model initialized with {model.count_parameters():,} parameters")

##  Implement and Compare Different Optimizers

**Task**: Set up multiple optimizers and compare their performance.

**Requirements**:
- Implement SGD, Adam, and AdamW optimizers
- Use appropriate hyperparameters for each
- Create a function to easily switch between optimizers

In [ ]:
def get_optimizer(model, optimizer_name, learning_rate):
    """
    Create optimizer based on name with appropriate hyperparameters

    Args:
        model: PyTorch model
        optimizer_name: String name of optimizer ('sgd', 'adam', 'adamw')
        learning_rate: Initial learning rate

    Returns:
        Configured optimizer
    """
    if optimizer_name.lower() == 'sgd':
        return optim.SGD(model.parameters(),
                        lr=learning_rate,
                        momentum=0.9,
                        weight_decay=1e-4)

    elif optimizer_name.lower() == 'adam':
        return optim.Adam(model.parameters(),
                         lr=learning_rate,
                         betas=(0.9, 0.999),
                         weight_decay=1e-4)

    elif optimizer_name.lower() == 'adamw':
        return optim.AdamW(model.parameters(),
                          lr=learning_rate,
                          betas=(0.9, 0.999),
                          weight_decay=1e-2)

    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

# Initialize optimizer - using AdamW as it generally provides better performance
optimizer = get_optimizer(model, 'adamw', LEARNING_RATE)
print(f"Optimizer configuration: {optimizer}")

## Use Learning Rate Scheduling

**Task**: Implement learning rate scheduling for improved training dynamics.

**Requirements**:
- Use StepLR or CosineAnnealingLR scheduler
- Configure appropriate scheduling parameters
- Track learning rate changes during training

In [ ]:
# Using CosineAnnealingLR for smooth learning rate decay
# This provides better convergence compared to step-based schedules
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

def get_current_lr(optimizer):
    """Get current learning rate from optimizer"""
    return optimizer.param_groups[0]['lr']

# Initialize list to track learning rates during training
learning_rates = []

print(f"Learning rate scheduler: CosineAnnealingLR")
print(f"Initial learning rate: {get_current_lr(optimizer)}")

## Apply Regularization Techniques  

**Task**: Implement various regularization methods to prevent overfitting.

**Requirements**:
- Use dropout in your model (already included in architecture)
- Implement early stopping mechanism
- Add L2 weight decay (already in optimizer)
- Optional: implement label smoothing

In [ ]:
class EarlyStopping:
    """Early stopping to prevent overfitting"""

    def __init__(self, patience=7, min_delta=0, restore_best_weights=True):

        self.patience = patience # Number of epochs to wait before stopping if no improvement
        self.min_delta = min_delta # Minimum change in the monitored quantity to qualify as an improvement
        self.restore_best_weights = restore_best_weights # Whether to restore model weights from the best epoch
        self.best_loss = None
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss, model):
        """
        Check if training should stop based on validation loss
        """
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1

        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False

    def save_checkpoint(self, model):
        """Save model weights"""
        self.best_weights = copy.deepcopy(model.state_dict())

# Initialize early stopping with patience of 10 epochs
early_stopping = EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

class LabelSmoothingCrossEntropy(nn.Module):
    """Label smoothing cross entropy loss for better generalization"""

    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, pred, target):
        """
        Args:
            pred: Predictions (batch_size, num_classes)
            target: Ground truth labels (batch_size,)
        """
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (pred.size(-1) - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))

# Use label smoothing for better generalization
criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
print(f"Loss function: Label Smoothing Cross Entropy (smoothing=0.1)")

##  Training Loop with Advanced Features

**Task**: Implement comprehensive training loop with all optimizations.

**Requirements**:
- Track multiple metrics during training
- Implement proper validation
- Save best model checkpoints
- Monitor learning rate and loss curves

In [ ]:
# Initialize tracking lists
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

def calculate_accuracy(outputs, labels):
    """Calculate accuracy from model outputs and true labels"""
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

def train_epoch(model, trainloader, criterion, optimizer, device):
    """Train model for one epoch"""
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total_batches = len(trainloader)

    for batch_idx, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track metrics
        running_loss += loss.item()
        running_acc += calculate_accuracy(outputs, labels)

        # Print progress
        if batch_idx % 100 == 0:
            print(f'Batch [{batch_idx}/{total_batches}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / total_batches
    epoch_acc = running_acc / total_batches
    return epoch_loss, epoch_acc

def validate_epoch(model, testloader, criterion, device):
    """Validate model for one epoch"""
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total_batches = len(testloader)

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            running_acc += calculate_accuracy(outputs, labels)

    epoch_loss = running_loss / total_batches
    epoch_acc = running_acc / total_batches
    return epoch_loss, epoch_acc

# Training loop
print("Starting training...")
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    print(f'\nEpoch [{epoch+1}/{NUM_EPOCHS}]')

    # Training phase
    train_loss, train_acc = train_epoch(model, trainloader, criterion, optimizer, device)

    # Validation phase
    val_loss, val_acc = validate_epoch(model, testloader, criterion, device)

    # Step scheduler
    scheduler.step()
    current_lr = get_current_lr(optimizer)

    # Track metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    learning_rates.append(current_lr)

    # Print epoch statistics
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print(f'Learning Rate: {current_lr:.6f}')

    # Check early stopping
    if early_stopping(val_loss, model):
        print(f'Early stopping triggered at epoch {epoch+1}')
        break

training_time = time.time() - start_time
print(f'\nTraining completed in {training_time:.2f} seconds')

# Plot training curves
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
ax1.plot(train_losses, label='Training Loss', color='blue')
ax1.plot(val_losses, label='Validation Loss', color='red')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

# Accuracy curves
ax2.plot(train_accuracies, label='Training Accuracy', color='blue')
ax2.plot(val_accuracies, label='Validation Accuracy', color='red')
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
ax2.grid(True)

# Learning rate schedule
ax3.plot(learning_rates, color='green')
ax3.set_title('Learning Rate Schedule')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Learning Rate')
ax3.grid(True)

# Training progress
epochs_run = len(train_losses)
ax4.plot(range(epochs_run), [max(val_accuracies[:i+1]) for i in range(epochs_run)],
         label='Best Validation Accuracy', color='purple')
ax4.set_title('Best Validation Accuracy Progress')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Best Val Accuracy (%)')
ax4.legend()
ax4.grid(True)

plt.tight_layout()
plt.show()


## Evaluate Model with Advanced Metrics

**Task**: Comprehensive evaluation using multiple metrics and visualizations.

**Requirements**:
- Calculate accuracy, precision, recall, F1-score
- Generate confusion matrix
- Analyze per-class performance
- Visualize misclassified examples

In [ ]:
def evaluate_model(model, testloader, device, class_names):
    """Comprehensive model evaluation"""
    model.eval()
    all_predictions = []
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(F.softmax(outputs, dim=1).cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision_macro = precision_score(all_labels, all_predictions, average='macro')
    recall_macro = recall_score(all_labels, all_predictions, average='macro')
    f1_macro = f1_score(all_labels, all_predictions, average='macro')

    precision_weighted = precision_score(all_labels, all_predictions, average='weighted')
    recall_weighted = recall_score(all_labels, all_predictions, average='weighted')
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted')

    # Print overall metrics
    print("==============================")
    print("EVALUATION RESULTS")
    print("==============================")
    print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)

    # Per-class accuracy
    per_class_acc = cm.diagonal() / cm.sum(axis=1)
    print("\nPer-class Accuracy:")
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: {per_class_acc[i]:.4f} ({per_class_acc[i]*100:.2f}%)")

    return {
        'predictions': all_predictions,
        'labels': all_labels,
        'outputs': all_outputs,
        'confusion_matrix': cm,
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'per_class_accuracy': per_class_acc
    }

# Evaluate the model
results = evaluate_model(model, testloader, device, CIFAR10_CLASSES)

# Generate classification report
print("\nDETAILED CLASSIFICATION REPORT")
print("=================================================================")
print(classification_report(results['labels'], results['predictions'],
                          target_names=CIFAR10_CLASSES, digits=4))

# Create confusion matrix visualization
plt.figure(figsize=(12, 10))
cm_normalized = results['confusion_matrix'].astype('float') / results['confusion_matrix'].sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues',
            xticklabels=CIFAR10_CLASSES, yticklabels=CIFAR10_CLASSES)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Raw confusion matrix with counts
plt.figure(figsize=(12, 10))
sns.heatmap(results['confusion_matrix'], annot=True, fmt='d', cmap='Oranges',
            xticklabels=CIFAR10_CLASSES, yticklabels=CIFAR10_CLASSES)
plt.title('Confusion Matrix (Raw Counts)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

def find_misclassified_examples(model, testloader, device, class_names, num_examples=20):
    """Find and return misclassified examples for visualization"""
    model.eval()
    misclassified = []

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            # Find misclassified examples
            wrong_indices = (predicted != labels).nonzero(as_tuple=True)[0]

            for idx in wrong_indices:
                if len(misclassified) >= num_examples:
                    break

                misclassified.append({
                    'image': inputs[idx].cpu(),
                    'true_label': labels[idx].item(),
                    'predicted_label': predicted[idx].item(),
                    'confidence': probabilities[idx][predicted[idx]].item(),
                    'true_confidence': probabilities[idx][labels[idx]].item()
                })

            if len(misclassified) >= num_examples:
                break

    return misclassified

# Find misclassified examples
misclassified_examples = find_misclassified_examples(model, testloader, device, CIFAR10_CLASSES, 16)

# Visualize misclassified examples
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
axes = axes.flatten()

for i, example in enumerate(misclassified_examples):
    if i >= 16:
        break

    # Unnormalize image for display
    img = example['image'].permute(1, 2, 0).numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)

    axes[i].imshow(img)
    true_class = CIFAR10_CLASSES[example['true_label']]
    pred_class = CIFAR10_CLASSES[example['predicted_label']]
    confidence = example['confidence']

    axes[i].set_title(f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.3f}',
                     color='red', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Misclassified Examples', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

# Analyze worst performing classes
print("\nWORST PERFORMING CLASSES")
print("=====================================")
worst_classes = np.argsort(results['per_class_accuracy'])[:3]
for idx in worst_classes:
    class_name = CIFAR10_CLASSES[idx]
    accuracy = results['per_class_accuracy'][idx]
    print(f"{class_name}: {accuracy:.4f} ({accuracy*100:.2f}%)")


##  Visualize Results

**Task**: Create comprehensive visualizations of model performance and behavior.

**Requirements**:
- Plot training/validation curves
- Visualize model predictions
- Show sample activations or feature maps
- Create performance comparison charts

In [ ]:
def plot_comprehensive_results(train_losses, val_losses, train_accs, val_accs,
                             learning_rates, results, class_names):
    """Create comprehensive visualization of all results"""

    fig = plt.figure(figsize=(20, 15))

    # Training curves
    ax1 = plt.subplot(3, 3, 1)
    plt.plot(train_losses, label='Training Loss', color='blue', linewidth=2)
    plt.plot(val_losses, label='Validation Loss', color='red', linewidth=2)
    plt.title('Loss Curves', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    ax2 = plt.subplot(3, 3, 2)
    plt.plot(train_accs, label='Training Accuracy', color='blue', linewidth=2)
    plt.plot(val_accs, label='Validation Accuracy', color='red', linewidth=2)
    plt.title('Accuracy Curves', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    ax3 = plt.subplot(3, 3, 3)
    plt.plot(learning_rates, color='green', linewidth=2)
    plt.title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.grid(True, alpha=0.3)

    # Per-class performance
    ax4 = plt.subplot(3, 3, 4)
    bars = plt.bar(range(len(class_names)), results['per_class_accuracy'],
                   color='skyblue', edgecolor='navy', alpha=0.7)
    plt.title('Per-Class Accuracy', fontsize=14, fontweight='bold')
    plt.xlabel('Class')
    plt.ylabel('Accuracy')
    plt.xticks(range(len(class_names)), class_names, rotation=45)
    plt.grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    for bar, acc in zip(bars, results['per_class_accuracy']):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{acc:.3f}', ha='center', va='bottom', fontsize=9)

    # Training progress metrics
    ax5 = plt.subplot(3, 3, 5)
    epochs = range(1, len(train_losses) + 1)
    best_val_acc = [max(val_accs[:i]) for i in range(1, len(val_accs) + 1)]
    plt.plot(epochs, best_val_acc, label='Best Val Accuracy', color='purple', linewidth=2)
    plt.plot(epochs, val_accs, label='Current Val Accuracy', color='orange', alpha=0.7)
    plt.title('Validation Accuracy Progress', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Loss difference (overfitting indicator)
    ax6 = plt.subplot(3, 3, 6)
    loss_diff = np.array(train_losses) - np.array(val_losses)
    plt.plot(loss_diff, color='red', linewidth=2)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.title('Training - Validation Loss\n(Overfitting Indicator)', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss Difference')
    plt.grid(True, alpha=0.3)

    # Final metrics summary
    ax7 = plt.subplot(3, 3, 7)
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    values = [results['accuracy'], results['precision_macro'],
              results['recall_macro'], results['f1_macro']]

    bars = plt.bar(metrics, values, color=['gold', 'lightcoral', 'lightblue', 'lightgreen'],
                   edgecolor='black', alpha=0.8)
    plt.title('Final Model Metrics', fontsize=14, fontweight='bold')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3, axis='y')

    for bar, val in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{val:.3f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

    # Confusion matrix (smaller version)
    ax8 = plt.subplot(3, 3, 8)
    cm_norm = results['confusion_matrix'].astype('float') / results['confusion_matrix'].sum(axis=1)[:, np.newaxis]
    im = plt.imshow(cm_norm, interpolation='nearest', cmap='Blues')
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.colorbar(im, fraction=0.046, pad=0.04)

    # Training time and efficiency
    ax9 = plt.subplot(3, 3, 9)
    final_train_acc = train_accs[-1]
    final_val_acc = val_accs[-1]
    best_val_acc_final = max(val_accs)

    categories = ['Final\nTrain Acc', 'Final\nVal Acc', 'Best\nVal Acc']
    acc_values = [final_train_acc, final_val_acc, best_val_acc_final]

    bars = plt.bar(categories, acc_values, color=['steelblue', 'darkorange', 'forestgreen'],
                   alpha=0.8, edgecolor='black')
    plt.title('Training Summary', fontsize=14, fontweight='bold')
    plt.ylabel('Accuracy (%)')
    plt.grid(True, alpha=0.3, axis='y')

    for bar, val in zip(bars, acc_values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

    plt.tight_layout()
    plt.show()

# Create comprehensive visualization
plot_comprehensive_results(train_losses, val_losses, train_accuracies, val_accuracies,
                          learning_rates, results, CIFAR10_CLASSES)

def visualize_predictions_with_confidence(model, testloader, device, class_names, num_samples=12):
    """Visualize model predictions with confidence scores"""
    model.eval()
    images_shown = 0
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            for i in range(inputs.size(0)):
                if images_shown >= num_samples:
                    break

                # Unnormalize image
                img = inputs[i].permute(1, 2, 0).cpu().numpy()
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                img = np.clip(img, 0, 1)

                axes[images_shown].imshow(img)

                true_label = labels[i].item()
                pred_label = predicted[i].item()
                confidence = probabilities[i][pred_label].item()

                # Color code: green for correct, red for incorrect
                color = 'green' if true_label == pred_label else 'red'

                title = f'True: {class_names[true_label]}\nPred: {class_names[pred_label]}\nConf: {confidence:.3f}'
                axes[images_shown].set_title(title, color=color, fontsize=10)
                axes[images_shown].axis('off')

                images_shown += 1

            if images_shown >= num_samples:
                break

    plt.suptitle('Model Predictions with Confidence Scores\n(Green: Correct, Red: Incorrect)',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize predictions with confidence
visualize_predictions_with_confidence(model, testloader, device, CIFAR10_CLASSES)

def visualize_feature_maps(model, testloader, device, layer_name='inception1'):
    """Visualize feature maps from a specific layer"""
    # Hook to capture feature maps
    feature_maps = []

    def hook_fn(module, input, output):
        feature_maps.append(output.cpu())

    # Register hook on the specified layer
    if layer_name == 'inception1':
        hook = model.inception1.register_forward_hook(hook_fn)
    elif layer_name == 'inception2':
        hook = model.inception2.register_forward_hook(hook_fn)
    else:
        print(f"Layer {layer_name} not found")
        return

    model.eval()
    with torch.no_grad():
        # Get one batch
        inputs, labels = next(iter(testloader))
        inputs = inputs.to(device)

        # Forward pass to trigger hook
        _ = model(inputs)

    # Remove hook
    hook.remove()

    # Visualize feature maps for first image
    if feature_maps:
        features = feature_maps[0][0]  # First image, all channels

        # Select first 16 channels to visualize
        num_features = min(16, features.shape[0])
        fig, axes = plt.subplots(4, 4, figsize=(12, 12))
        axes = axes.flatten()

        for i in range(num_features):
            feature_map = features[i].numpy()
            axes[i].imshow(feature_map, cmap='viridis')
            axes[i].set_title(f'Channel {i}', fontsize=10)
            axes[i].axis('off')

        # Hide unused subplots
        for i in range(num_features, 16):
            axes[i].axis('off')

        plt.suptitle(f'Feature Maps from {layer_name}', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

# Visualize feature maps from inception1 layer
visualize_feature_maps(model, testloader, device, 'inception1')

# Model architecture summary
def print_model_summary(model, input_size=(3, 32, 32)):
    """Print detailed model summary"""
    print("\n" + "===============================")
    print("MODEL ARCHITECTURE SUMMARY")
    print("==================================")

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Model Size (MB): {total_params * 4 / (1024 * 1024):.2f}")

    # Calculate model complexity
    dummy_input = torch.randn(1, *input_size).to(device)
    model.eval()

    with torch.no_grad():
        output = model(dummy_input)

    print(f"Input Shape: {input_size}")
    print(f"Output Shape: {output.shape[1:]}")
    print(f"Number of Classes: {output.shape[1]}")

    print("\nLayer-wise Parameter Count:")
    print("--------------------------------------")
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # Leaf modules only
            params = sum(p.numel() for p in module.parameters())
            if params > 0:
                print(f"{name:30} {params:>10,}")

print_model_summary(model)

# Final results summary
print("\nFINAL TRAINING SUMMARY")
print("========================================================")
print(f"Training completed in {len(train_losses)} epochs")
print(f"Best validation accuracy: {max(val_accuracies):.2f}%")
print(f"Final validation accuracy: {val_accuracies[-1]:.2f}%")
print(f"Test accuracy: {results['accuracy']*100:.2f}%")
print(f"Total training time: {training_time:.2f} seconds")
print(f"Average time per epoch: {training_time/len(train_losses):.2f} seconds")

# Check if target accuracy achieved
target_accuracy = 80.0
if results['accuracy'] * 100 >= target_accuracy:
  print("\n")
else:
    print(f"\nTarget accuracy of {target_accuracy}% not achieved.")
    print(f"Your model achieved {results['accuracy']*100:.2f}% accuracy")

print(f"Final model performance: {results['accuracy']*100:.2f}% accuracy")

# Save final model
torch.save(model.state_dict(), 'inception_cnn_cifar10.pth')
print(f"\nModel saved as 'inception_cnn_cifar10.pth'")
